## Libraries

In [73]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Understanding the Dataset

### Loading the Dataset

In [74]:
data = pd.read_csv('../Data/dataset_churn.csv')

data.head()

,Unnamed: 0,CustomerID,Age,Gender,Tenure,Service_Internet,Service_Phone,Service_TV,Contract,PaymentMethod,MonthlyCharges,TotalCharges,StreamingMovies,StreamingMusic,OnlineSecurity,TechSupport,Churn
0,0,08729464-bde6-43bc-8f63-a357096feab1,56.0,Male,13,DSL,Yes,No,One year,Mailed check,71.88,931.49,No,No,Yes,No,No
1,1,af95bc95-baf4-4318-a21d-70d2ea3148b7,69.0,Male,13,DSL,No,Yes,Two year,Mailed check,110.99,1448.46,Yes,Yes,No,No,No
2,2,1fe7eee6-2227-4400-9998-4d993f4a60fd,46.0,Male,60,Fiber optic,No,Yes,Month-to-month,Mailed check,116.74,6997.73,Yes,Yes,No,No,No
3,3,f736fe7b-1b44-4acd-84c2-21c4aef648be,32.0,Female,57,Fiber optic,Yes,Yes,Month-to-month,Bank transfer,78.16,4452.13,No,Yes,No,Yes,No
4,4,4b40d12d-7633-4309-96b8-aee675ea20ae,60.0,Male,52,Fiber optic,Yes,Yes,Two year,Electronic check,30.33,1569.73,Yes,No,Yes,Yes,No


In [75]:
# removing the index column from the dataset.
data = data.drop('Unnamed: 0', axis=1)

### Understanding the Dataset.

In [76]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3749 entries, 0 to 3748
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   CustomerID        3749 non-null   object 
 1   Age               3562 non-null   float64
 2   Gender            3749 non-null   object 
 3   Tenure            3749 non-null   int64  
 4   Service_Internet  3028 non-null   object 
 5   Service_Phone     3749 non-null   object 
 6   Service_TV        3749 non-null   object 
 7   Contract          3749 non-null   object 
 8   PaymentMethod     3562 non-null   object 
 9   MonthlyCharges    3749 non-null   float64
 10  TotalCharges      3749 non-null   float64
 11  StreamingMovies   3749 non-null   object 
 12  StreamingMusic    3749 non-null   object 
 13  OnlineSecurity    3749 non-null   object 
 14  TechSupport       3749 non-null   object 
 15  Churn             3749 non-null   object 
dtypes: float64(3), int64(1), object(12)
memory

In [77]:
data.describe()

,Age,Tenure,MonthlyCharges,TotalCharges
count,3562.000000,3749.000000,3749.000000,3749.000000
mean,43.655531,36.264070,75.844318,2718.968266
std,14.914474,20.505528,73.062971,3211.879149
min,18.000000,1.000000,20.000000,13.190000
25%,31.000000,19.000000,44.570000,1076.240000
50%,44.000000,36.000000,69.590000,2132.260000
75%,56.000000,54.000000,95.540000,3619.710000
max,69.000000,71.000000,1179.300000,79951.800000


In [78]:
# Checking for missing values
data.isnull().sum()

CustomerID            0
Age                 187
Gender                0
Tenure                0
Service_Internet    721
Service_Phone         0
Service_TV            0
Contract              0
PaymentMethod       187
MonthlyCharges        0
TotalCharges          0
StreamingMovies       0
StreamingMusic        0
OnlineSecurity        0
TechSupport           0
Churn                 0
dtype: int64